In [55]:
import pandas as pd
import openai
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import ast
import os
from dotenv import load_dotenv
import time

In [56]:
load_dotenv()
openai.api_key = os.environ['OPENAI_API']

In [57]:
def create_clean_paragraphs(input_dict):

    # Create a new dictionary to store the cleaned-up values
    output_dict = {}
    
    # Loop through the keys and values of the input dictionary
    for key, value in input_dict.items():
        # Convert the list of values to a set to remove duplicates
        unique_values = set(value)
        
        # Join the sentences together into a single string
        combined_string = ' '.join(unique_values)
        
        # Add the cleaned-up string to the output dictionary
        output_dict[key] = combined_string
    
    # Return the cleaned-up dictionary
    return output_dict

In [58]:
folder_survey = "../../../data/surveys/"
file = "df_survey_result.csv"
df = pd.read_csv(F"{folder_survey}{file}")
df = df.set_index("Species")

In [59]:
df_car = df[df['Dataset'] == 'Caribbean'].reset_index()
df_car

,Species,Unnamed: 0,Main Trait,SIM,Dataset,Sentence,Result
0,Bursera simaruba,451,Leaf glands,Jacc,Caribbean,"Leaf venation: brachidodrome, pinnate.",Can infer correct Entity
1,Bursera simaruba,452,Leaf glands,Jacc,Caribbean,"Leaf venation: brachidodrome, pinnate.",Can infer correct Entity
2,Bursera simaruba,453,Leaf rachis,Jacc,Caribbean,"Leaf venation: brachidodrome, pinnate.",Can infer correct Entity
3,Bursera simaruba,454,Leaf rachis,Jacc,Caribbean,"Leaf venation: brachidodrome, pinnate.",Can infer correct Entity
4,Crossopetalum rhacoma,455,Fruit type,Jacc,Caribbean,"Fruit a drupe, seed exarillate.",Can infer correct Quality
...,...,...,...,...,...,...,...
523,Hippomane mancinella,974,Leaf shape,Bert,Caribbean,"The leaves are arranged alternately, to 12 cm ...",Can infer correct Value
524,Bourreria succulenta,975,Petals / corolla number,Jacc,Caribbean,The calyx has 5 fused sepals forming a tube.,None of the above
525,Trichilia trifolia,976,Petals / corolla colour,Bert,Caribbean,"Nectary patelliform or cyathiform, usually fus...",None of the above
526,Clusia rosea,977,Leaf lower side,Jacc,Caribbean,"Leaf color: dull green on top, yellow green un...",Can infer correct Entity


In [81]:
print(df_car.loc[313, ])
print(df_car.loc[313, ].Sentence)

Species                                    Hippomane mancinella
Unnamed: 0                                                  764
Main Trait                              Petals / corolla colour
SIM                                                        Bert
Dataset                                               Caribbean
Sentence      Surrounding the carpellate flowers are glands ...
Result                                 Can infer correct Entity
Name: 313, dtype: object
Surrounding the carpellate flowers are glands and often-additional staminate flowers.


In [77]:
sentence = df_car.loc[111, 'Sentence']
trait = df_car.loc[111, 'Main Trait']

user_content = F"""
Can you help us with determining if, using only the provided text, it would be possible to infer the entity, quality or value from the sentence for the trait: '{trait}'?

 There are four options:
	- Can infer correct Entity
	- Can infer correct Quality
	- Can infer correct Value
	- None of the above

For each choice we have a small example:
	- Correct Entity: e.g. crown, flower or fruit.
	- Correct Quality: e.g. forest layer, number of petals or colour.
	- Correct Value: e.g. canopy, 5 or red.
"""

In [78]:
print(sentence)
print(user_content)

Leaf stems are yellow-green.

Can you help us with determining if, using only the provided text, it would be possible to infer the entity, quality or value from the sentence for the trait: 'Leaf glands'?

 There are four options:
	- Can infer correct Entity
	- Can infer correct Quality
	- Can infer correct Value
	- None of the above

For each choice we have a small example:
	- Correct Entity: e.g. crown, flower or fruit.
	- Correct Quality: e.g. forest layer, number of petals or colour.
	- Correct Value: e.g. canopy, 5 or red.



In [79]:
messages = [
    {"role": "user", "content": sentence},
    {"role": "user", "content": user_content}
    ]
# Call the ChatGPT API to generate a completion for the prompt
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = messages,
)

In [80]:
completion

<OpenAIObject chat.completion id=chatcmpl-6zonZDKHhjS0fpSe7rNcwAzJORleX at 0x7fcc2aa7ddf0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "None of the above. The provided sentence only mentions the stem color of the leaves, and there is no mention of any gland related to the leaves.",
        "role": "assistant"
      }
    }
  ],
  "created": 1680191297,
  "id": "chatcmpl-6zonZDKHhjS0fpSe7rNcwAzJORleX",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 30,
    "prompt_tokens": 146,
    "total_tokens": 176
  }
}

In [ ]:
# Define the path to the directory where the prompts and results will be saved
folder_prompts = "../../../data/OpenAI/PromptsResults/ZeroShot/"

species_lst = df_Andrei.index
trait_dict = create_trait_dict(df_Andrei)
traits = list(trait_dict.keys())

# Loop over species
for idx, species in enumerate(species_lst[0:]):
   
    # Replace spaces in the species name with underscores
    folder_species = species.replace(' ', '_')

    # Try to create a directory for the prompts for the species
    try:
        os.makedirs(F"{folder_prompts}{folder_species}")
    except FileExistsError:
        pass

    for trait in (pbar := tqdm(traits[0:], leave=False, position=0)):
        pbar.set_description(f"{idx}: {species}")

        # Get the options for the trait
        options = trait_dict[trait]

        # Create the question and options for the ChatGPT prompt
        question = F"For the species '{species}' which of the following options is applicable for the trait '{trait}'" 
        text_helper = F"Return 'NaN' if none of the options are applicable. Please return your answer as a Python list."
        user_content = F"{question}\n{options}\n{text_helper}"

        # Combine the words in the trait name with capital letters and use this as the file name
        file_name = combine_words_with_capital(trait)
        
        # Check if file is already there (OpenAI Outage)
        if os.path.exists(F"{folder_prompts}{folder_species}/{file_name}.json"):
            continue

        # # Create the messages to send to the ChatGPT API
        messages = [
            {"role": "user", "content": sentence},
            {"role": "user", "content": user_content}
            ]
        # Call the ChatGPT API to generate a completion for the prompt
        completion = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = messages,
        )

        # Lower pressure on API?
        time.sleep(2)

        # Save the completion to a JSON file with the file name in the directory
        with open(F"{folder_prompts}{folder_species}/{file_name}.json", 'w') as fp:
            json.dump(completion, fp)
            